In [2]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 2
setup_one_gpu(GPU)

Picking GPU 2


In [70]:
import tensorflow as tf
import warnings
import os.path as osp
import numpy as np
from tf_lab.evaluate.generative_pc_nets import minimum_mathing_distance

In [42]:
n_examples = 10

In [43]:
class_name = 'chair'
top_gt_dir = '/orions4-zfs/projects/optas/DATA/OUT/iclr/evaluations/gt_data/'
gt_train_file = osp.join(top_gt_dir, class_name + '_train.npz')
gt_train_data = np.load(gt_train_file)  
gt_train_data = gt_train_data[gt_train_data.keys()[0]]

In [44]:
a_pc_set = gt_train_data[:n_examples]
b_pc_set = gt_train_data[n_examples:n_examples*2]

In [45]:
from geo_tool.point_clouds.aux import chamfer_pseudo_distance

In [110]:
mmd_dists = []
for i in range(n_examples):
    all_dist_i = []
    for b in b_pc_set:
        all_dist_i.append(chamfer_pseudo_distance(a_pc_set[i], b))
    mmd_dists.append(min(all_dist_i))

In [111]:
np.mean(mmd_dists)

162.45324374641538

In [107]:
def minimum_mathing_distance_3(sample_pcs, ref_pcs, batch_size, normalize=False, sess=None, verbose=False):
    ''' normalize (boolean): if True the Chamfer distance between two point-clouds is the average of matched
                             point-distances. Alternatively, is their sum.
    '''
    if normalize:
        reducer = tf.reduce_mean
    else:
        reducer = tf.reduce_sum

    if sess is None:
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        sess = tf.Session(config=config)

    n_ref, n_pc_points, pc_dim = ref_pcs.shape
    _, n_pc_points_s, pc_dim_s = sample_pcs.shape

    if n_pc_points != n_pc_points_s or pc_dim != pc_dim_s:
        raise ValueError('Incompatible Point-Clouds.')

    # TF Graph Operations
    ref_pl = tf.placeholder(tf.float32, shape=(1, n_pc_points, pc_dim))
    sample_pl = tf.placeholder(tf.float32, shape=(None, n_pc_points, pc_dim))

    repeat_times = tf.shape(sample_pl)[0]   # slower- could be used to use entire set of samples.
#     repeat_times = batch_size
    ref_repeat = tf.tile(ref_pl, [repeat_times, 1, 1])
    ref_repeat = tf.reshape(ref_repeat, [repeat_times, n_pc_points, pc_dim])

    ref_to_s, _, s_to_ref, _ = nn_distance(ref_repeat, sample_pl)
    ref_to_s = tf.sqrt(ref_to_s)
    s_to_ref = tf.sqrt(s_to_ref)
    
    chamfer_dist_batch = reducer(ref_to_s, 1) + reducer(s_to_ref, 1)

    best_in_batch = tf.reduce_min(chamfer_dist_batch)   # Best distance, of those that were matched to single ref pc.
    matched_dists = []
    for i in xrange(n_ref):
        best_in_all_batches = []
        if verbose and i % 50 == 0:
            print i
        for sample_chunk in iterate_in_chunks(sample_pcs, batch_size):
#             if len(sample_chunk) != batch_size:
#                 continue
            feed_dict = {ref_pl: np.expand_dims(ref_pcs[i], 0), sample_pl: sample_chunk}
            b = sess.run(best_in_batch, feed_dict=feed_dict)
            best_in_all_batches.append(b)

        matched_dists.append(np.min(best_in_all_batches))

    mmd = np.mean(matched_dists)
    return mmd, matched_dists

In [114]:
minimum_mathing_distance_3(b_pc_set, a_pc_set, 3)

(162.45323,
 [209.44739,
  171.06543,
  185.34439,
  150.00891,
  174.49524,
  143.40103,
  164.32225,
  140.96434,
  135.12698,
  150.35646])